In [38]:
from os import path
import json

import numpy as np

In [39]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [40]:
base_data_dir = '/home/shtoshni/Research/litbank_coref/data'
data_dir_dict = {
    'ontonotes': path.join(base_data_dir, 'ontonotes/all.jsonlines'),
    'litbank': path.join(base_data_dir, f'litbank/all.jsonlines'),
    'preco': path.join(base_data_dir, 'preco/all.jsonlines'),
    'character_identification': path.join(base_data_dir, 'character_identification/all.jsonlines'),
    'wikicoref': path.join(base_data_dir, 'wikicoref/all.jsonlines'),
    'quizbowl': path.join(base_data_dir, 'quizbowl/all.jsonlines'),
    'gap': path.join(base_data_dir, 'gap/all.jsonlines'),

}


In [41]:
for dataset, data_file in data_dir_dict.items():
    num_tokens_list = []
    num_mentions_list = []
    mention_lens_list = []
    cluster_lens_list = []
    num_singleton_mentions = []
    
    with open(data_file) as f:
        for line in f:
            instance = json.loads(line.strip())
            num_tokens = instance['subtoken_map'][-1] - instance['subtoken_map'][0] + 1
            num_tokens_list.append(num_tokens)
            
            if dataset != 'gap':
                mentions = flatten(instance['clusters'])
                num_mentions = len(mentions)
                num_mentions_list.append(num_mentions)
            
                mention_lens = [(
                    instance['subtoken_map'][ment_end] - instance['subtoken_map'][ment_start] + 1)
                    for ment_start, ment_end in mentions
                ]
                cluster_lens = [len(cluster) for cluster in instance['clusters']]
                num_singleton_mentions.append(
                    len([cluster for cluster in instance['clusters'] if len(cluster) == 1])
                )

                mention_lens_list.extend(mention_lens)
                cluster_lens_list.extend(cluster_lens)
                
            else:
                mention_lens = []
                for key in ['pronoun_span', 'a_span', 'b_span']:
                    ment_start, ment_end = instance[key]
                    mention_lens.append(
                        instance['subtoken_map'][ment_end] - instance['subtoken_map'][ment_start] + 1)
                    
                mention_lens_list.extend(mention_lens)
            
            
    print(f'{dataset} & {np.mean(num_tokens_list):.0f} & {np.mean(num_mentions_list):.0f} '
          f'& {np.mean(mention_lens_list):.1f} & {np.mean(cluster_lens_list):.1f} & {(np.sum(num_singleton_mentions) * 100.0)/np.sum(num_mentions_list):.1f}'
         )
    
            
            
            
    

ontonotes & 467 & 56 & 2.3 & 4.4 & 0.0
litbank & 2105 & 291 & 2.0 & 3.7 & 19.8
preco & 337 & 105 & 2.7 & 1.6 & 52.0
character_identification & 262 & 36 & 1.0 & 5.1 & 6.4
wikicoref & 1996 & 230 & 2.6 & 5.0 & 0.0
quizbowl & 126 & 24 & 2.7 & 2.0 & 26.0
gap & 95 & nan & 2.0 & nan & nan


<ipython-input-41-50c7bf646ef5>:41: RuntimeWarning: invalid value encountered in double_scalars
  print(f'{dataset} & {np.mean(num_tokens_list):.0f} & {np.mean(num_mentions_list):.0f} '
